# Part 1: segmenting the cities in neighborhoods

Packages required

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import geocoder

import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
import requests

from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

import folium

import os
from dotenv import load_dotenv

chicago neighborhoods

In [2]:
chicago_url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
source = requests.get(chicago_url).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
body = table.find('tbody')
tsas = []
for tsa in table.find_all('tr'):
    tsas.append (tsa.get_text().splitlines()[1])
tsas = tsas [1:]
labels = ['Neighborhood']
chicago_neighborhoods = pd.DataFrame (tsas, columns=labels)

Latitudes=[]
Longitudes=[]
for i in range(chicago_neighborhoods.shape[0]):
    arg='{}, Chicago, Illinois'.format(chicago_neighborhoods.loc[i,'Neighborhood'])
    geopoint = geocoder.arcgis(arg)
    Latitudes.append(geopoint.latlng[0])
    Longitudes.append(geopoint.latlng[1])
    
chicago_neighborhoods['Latitude']=Latitudes
chicago_neighborhoods['Longitude']=Longitudes

ny neighborhoods

In [3]:
with open('ny_neighborhoods.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
ny_neighborhoods = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'])
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipython-input-3-daf806102741>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
<ipy

In [4]:
ny_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Wakefield,40.894705,-73.847201
1,Co-op City,40.874294,-73.829939
2,Eastchester,40.887556,-73.827806
3,Fieldston,40.895437,-73.905643
4,Riverdale,40.890834,-73.912585


In [5]:
chicago_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Albany Park,41.96829,-87.72338
1,Altgeld Gardens,41.65448,-87.60225
2,Andersonville,41.98046,-87.66834
3,Arcadia Terrace,41.55990,-87.55627
4,Archer Heights,41.81154,-87.72556


# Part 2: getting relevant data from Foursquare API

Foursquare credentials

In [6]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
VERSION = os.getenv('VERSION')

method to get nearby venues

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

getting nearby venues for the neighborhoods in the 2 cities

In [8]:
ny_venues=getNearbyVenues(names=ny_neighborhoods['Neighborhood'], latitudes=ny_neighborhoods['Latitude'], longitudes=ny_neighborhoods['Longitude'])
chicago_venues=getNearbyVenues(names=chicago_neighborhoods['Neighborhood'], latitudes=chicago_neighborhoods['Latitude'], longitudes=chicago_neighborhoods['Longitude'])

In [9]:
ny_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


one hot encoding per venue type

In [10]:
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
ny_onehot['Neighborhood'] = ny_venues['Neighborhood']

chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood']

# only keep categories present in both cities
fixed_columns = list(set(chicago_onehot.columns) & set(ny_onehot.columns))

ny_onehot = ny_onehot[fixed_columns]
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_total = ny_venues.groupby(['Neighborhood']).size().reset_index(name='counts')
ny_data = pd.merge(ny_grouped, ny_total, on='Neighborhood')

chicago_onehot = chicago_onehot[fixed_columns]
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()
chicago_total = chicago_venues.groupby(['Neighborhood']).size().reset_index(name='counts')
chicago_data = pd.merge(chicago_grouped, chicago_total, on='Neighborhood')

In [11]:
chicago_data.head()

,Neighborhood,Department Store,Electronics Store,Pub,Museum,Breakfast Spot,Diner,Creperie,Irish Pub,Toy / Game Store,Pakistani Restaurant,Boxing Gym,Polish Restaurant,Insurance Office,Rental Car Location,Athletics & Sports,Farm,Health & Beauty Service,South American Restaurant,Fast Food Restaurant,Burmese Restaurant,Miscellaneous Shop,Russian Restaurant,Spanish Restaurant,Bar,Outdoor Sculpture,Malay Restaurant,Martial Arts School,Automotive Shop,Tanning Salon,Tea Room,Cosmetics Shop,Thai Restaurant,Bagel Shop,Food,Business Service,Paper / Office Supplies Store,Fish Market,Molecular Gastronomy Restaurant,Garden Center,Market,Basketball Court,Concert Hall,Donut Shop,Men's Store,Shipping Store,Chocolate Shop,Design Studio,Gay Bar,Boutique,Mattress Store,Fruit & Vegetable Store,Dance Studio,Vegetarian / Vegan Restaurant,Skate Park,Board Shop,Liquor Store,Trail,Weight Loss Center,Hostel,Heliport,Mobile Phone Shop,Bike Shop,Gym / Fitness Center,Butcher,Piercing Parlor,Bowling Alley,Opera House,Clothing Store,Baseball Stadium,Doctor's Office,ATM,Food & Drink Shop,Bistro,Print Shop,Veterinarian,Skating Rink,Discount Store,Antique Shop,Sporting Goods Shop,Tex-Mex Restaurant,Gym Pool,Non-Profit,Flea Market,Park,Video Store,Argentinian Restaurant,Arts & Entertainment,Building,Eastern European Restaurant,Fried Chicken Joint,Shop & Service,Hobby Shop,Lounge,Train Station,Lingerie Store,Intersection,Residential Building (Apartment / Condo),Construction & Landscaping,Other Great Outdoors,Soccer Field,Tattoo Parlor,Peruvian Restaurant,Bike Rental / Bike Share,Pilates Studio,Chinese Restaurant,Flower Shop,Accessories Store,Sandwich Place,Gas Station,Latin American Restaurant,Shoe Store,Health Food Store,Whisky Bar,Cajun / Creole Restaurant,Jazz Club,Czech Restaurant,Burrito Place,Thrift / Vintage Store,Comedy Club,Hawaiian Restaurant,Pharmacy,Baseball Field,Art Gallery,Pizza Place,Bakery,Moving Target,Poke Place,Dog Run,Sports Bar,IT Services,Roof Deck,Café,Bus Stop,Hardware Store,Lake,Women's Store,Middle Eastern Restaurant,Beer Garden,Food Court,Japanese Restaurant,Jewelry Store,Cupcake Shop,Other Nightlife,Bookstore,Art Museum,Storage Facility,Hookah Bar,Supplement Shop,Buffet,History Museum,Halal Restaurant,French Restaurant,Mediterranean Restaurant,Ethiopian Restaurant,Big Box Store,Coffee Shop,Pie Shop,Photography Studio,Video Game Store,Smoothie Shop,Tennis Court,Farmers Market,Rock Club,Hotel Bar,Indie Theater,Convenience Store,Theater,Falafel Restaurant,Road,Ramen Restaurant,Harbor / Marina,Baby Store,Historic Site,Outlet Store,American Restaurant,Tiki Bar,Supermarket,Salon / Barbershop,Beach,Cafeteria,Hot Dog Joint,Seafood Restaurant,Turkish Restaurant,Bike Trail,Animal Shelter,Hotel,Bridal Shop,Lawyer,Persian Restaurant,Dry Cleaner,Taco Place,Steakhouse,Climbing Gym,Airport Terminal,Scandinavian Restaurant,Bus Line,Music Store,Speakeasy,Southern / Soul Food Restaurant,Wine Shop,Field,New American Restaurant,Sushi Restaurant,Food Stand,Indian Restaurant,Afghan Restaurant,Snack Place,Outdoor Supply Store,African Restaurant,Motorcycle Shop,Noodle House,Gift Shop,Beer Bar,Sculpture Garden,Pet Service,Korean Restaurant,Shopping Mall,Bank,Greek Restaurant,Deli / Bodega,Multiplex,Hotpot Restaurant,Indie Movie Theater,Home Service,Costume Shop,Rental Service,Wine Bar,Filipino Restaurant,Cuban Restaurant,Metro Station,Yemeni Restaurant,Gym,Music Venue,Spa,Street Art,Indoor Play Area,Furniture / Home Store,BBQ Joint,Cocktail Bar,Empanada Restaurant,North Indian Restaurant,Comfort Food Restaurant,Bubble Tea Shop,General Entertainment,Kids Store,Performing Arts Venue,Yoga Studio,Nightclub,Playground,Burger Joint,Brazilian Restaurant,Smoke Shop,Taiwanese Restaurant,College Cafeteria,Dessert Shop,Dentist's Office,Pool,Shopping Plaza,Karaoke Bar,Massage Studio,Salad Place,Szechuan Restaurant,Israeli Restaurant,Check Cashing Service,Recreation Center,Other Repair Shop,Dive Bar,Tailor Shop,Track,Gourmet Shop,Candy Store,Gastropub,Boat or Ferry,Laund

In [12]:
ny_data.head()

,Neighborhood,Department Store,Electronics Store,Pub,Museum,Breakfast Spot,Diner,Creperie,Irish Pub,Toy / Game Store,Pakistani Restaurant,Boxing Gym,Polish Restaurant,Insurance Office,Rental Car Location,Athletics & Sports,Farm,Health & Beauty Service,South American Restaurant,Fast Food Restaurant,Burmese Restaurant,Miscellaneous Shop,Russian Restaurant,Spanish Restaurant,Bar,Outdoor Sculpture,Malay Restaurant,Martial Arts School,Automotive Shop,Tanning Salon,Tea Room,Cosmetics Shop,Thai Restaurant,Bagel Shop,Food,Business Service,Paper / Office Supplies Store,Fish Market,Molecular Gastronomy Restaurant,Garden Center,Market,Basketball Court,Concert Hall,Donut Shop,Men's Store,Shipping Store,Chocolate Shop,Design Studio,Gay Bar,Boutique,Mattress Store,Fruit & Vegetable Store,Dance Studio,Vegetarian / Vegan Restaurant,Skate Park,Board Shop,Liquor Store,Trail,Weight Loss Center,Hostel,Heliport,Mobile Phone Shop,Bike Shop,Gym / Fitness Center,Butcher,Piercing Parlor,Bowling Alley,Opera House,Clothing Store,Baseball Stadium,Doctor's Office,ATM,Food & Drink Shop,Bistro,Print Shop,Veterinarian,Skating Rink,Discount Store,Antique Shop,Sporting Goods Shop,Tex-Mex Restaurant,Gym Pool,Non-Profit,Flea Market,Park,Video Store,Argentinian Restaurant,Arts & Entertainment,Building,Eastern European Restaurant,Fried Chicken Joint,Shop & Service,Hobby Shop,Lounge,Train Station,Lingerie Store,Intersection,Residential Building (Apartment / Condo),Construction & Landscaping,Other Great Outdoors,Soccer Field,Tattoo Parlor,Peruvian Restaurant,Bike Rental / Bike Share,Pilates Studio,Chinese Restaurant,Flower Shop,Accessories Store,Sandwich Place,Gas Station,Latin American Restaurant,Shoe Store,Health Food Store,Whisky Bar,Cajun / Creole Restaurant,Jazz Club,Czech Restaurant,Burrito Place,Thrift / Vintage Store,Comedy Club,Hawaiian Restaurant,Pharmacy,Baseball Field,Art Gallery,Pizza Place,Bakery,Moving Target,Poke Place,Dog Run,Sports Bar,IT Services,Roof Deck,Café,Bus Stop,Hardware Store,Lake,Women's Store,Middle Eastern Restaurant,Beer Garden,Food Court,Japanese Restaurant,Jewelry Store,Cupcake Shop,Other Nightlife,Bookstore,Art Museum,Storage Facility,Hookah Bar,Supplement Shop,Buffet,History Museum,Halal Restaurant,French Restaurant,Mediterranean Restaurant,Ethiopian Restaurant,Big Box Store,Coffee Shop,Pie Shop,Photography Studio,Video Game Store,Smoothie Shop,Tennis Court,Farmers Market,Rock Club,Hotel Bar,Indie Theater,Convenience Store,Theater,Falafel Restaurant,Road,Ramen Restaurant,Harbor / Marina,Baby Store,Historic Site,Outlet Store,American Restaurant,Tiki Bar,Supermarket,Salon / Barbershop,Beach,Cafeteria,Hot Dog Joint,Seafood Restaurant,Turkish Restaurant,Bike Trail,Animal Shelter,Hotel,Bridal Shop,Lawyer,Persian Restaurant,Dry Cleaner,Taco Place,Steakhouse,Climbing Gym,Airport Terminal,Scandinavian Restaurant,Bus Line,Music Store,Speakeasy,Southern / Soul Food Restaurant,Wine Shop,Field,New American Restaurant,Sushi Restaurant,Food Stand,Indian Restaurant,Afghan Restaurant,Snack Place,Outdoor Supply Store,African Restaurant,Motorcycle Shop,Noodle House,Gift Shop,Beer Bar,Sculpture Garden,Pet Service,Korean Restaurant,Shopping Mall,Bank,Greek Restaurant,Deli / Bodega,Multiplex,Hotpot Restaurant,Indie Movie Theater,Home Service,Costume Shop,Rental Service,Wine Bar,Filipino Restaurant,Cuban Restaurant,Metro Station,Yemeni Restaurant,Gym,Music Venue,Spa,Street Art,Indoor Play Area,Furniture / Home Store,BBQ Joint,Cocktail Bar,Empanada Restaurant,North Indian Restaurant,Comfort Food Restaurant,Bubble Tea Shop,General Entertainment,Kids Store,Performing Arts Venue,Yoga Studio,Nightclub,Playground,Burger Joint,Brazilian Restaurant,Smoke Shop,Taiwanese Restaurant,College Cafeteria,Dessert Shop,Dentist's Office,Pool,Shopping Plaza,Karaoke Bar,Massage Studio,Salad Place,Szechuan Restaurant,Israeli Restaurant,Check Cashing Service,Recreation Center,Other Repair Shop,Dive Bar,Tailor Shop,Track,Gourmet Shop,Candy Store,Gastropub,Boat or Ferry,Laund

only keep neighborhoods with more than 20 venues

In [13]:
chicago_30 = chicago_data[chicago_data['counts'] > 30]
ny_30 = ny_data[ny_data['counts'] > 30]

In [14]:
chicago_30.set_index('Neighborhood', inplace=True)
ny_30.set_index('Neighborhood', inplace=True)

In [15]:
ny_30.head()

,Department Store,Electronics Store,Pub,Museum,Breakfast Spot,Diner,Creperie,Irish Pub,Toy / Game Store,Pakistani Restaurant,Boxing Gym,Polish Restaurant,Insurance Office,Rental Car Location,Athletics & Sports,Farm,Health & Beauty Service,South American Restaurant,Fast Food Restaurant,Burmese Restaurant,Miscellaneous Shop,Russian Restaurant,Spanish Restaurant,Bar,Outdoor Sculpture,Malay Restaurant,Martial Arts School,Automotive Shop,Tanning Salon,Tea Room,Cosmetics Shop,Thai Restaurant,Bagel Shop,Food,Business Service,Paper / Office Supplies Store,Fish Market,Molecular Gastronomy Restaurant,Garden Center,Market,Basketball Court,Concert Hall,Donut Shop,Men's Store,Shipping Store,Chocolate Shop,Design Studio,Gay Bar,Boutique,Mattress Store,Fruit & Vegetable Store,Dance Studio,Vegetarian / Vegan Restaurant,Skate Park,Board Shop,Liquor Store,Trail,Weight Loss Center,Hostel,Heliport,Mobile Phone Shop,Bike Shop,Gym / Fitness Center,Butcher,Piercing Parlor,Bowling Alley,Opera House,Clothing Store,Baseball Stadium,Doctor's Office,ATM,Food & Drink Shop,Bistro,Print Shop,Veterinarian,Skating Rink,Discount Store,Antique Shop,Sporting Goods Shop,Tex-Mex Restaurant,Gym Pool,Non-Profit,Flea Market,Park,Video Store,Argentinian Restaurant,Arts & Entertainment,Building,Eastern European Restaurant,Fried Chicken Joint,Shop & Service,Hobby Shop,Lounge,Train Station,Lingerie Store,Intersection,Residential Building (Apartment / Condo),Construction & Landscaping,Other Great Outdoors,Soccer Field,Tattoo Parlor,Peruvian Restaurant,Bike Rental / Bike Share,Pilates Studio,Chinese Restaurant,Flower Shop,Accessories Store,Sandwich Place,Gas Station,Latin American Restaurant,Shoe Store,Health Food Store,Whisky Bar,Cajun / Creole Restaurant,Jazz Club,Czech Restaurant,Burrito Place,Thrift / Vintage Store,Comedy Club,Hawaiian Restaurant,Pharmacy,Baseball Field,Art Gallery,Pizza Place,Bakery,Moving Target,Poke Place,Dog Run,Sports Bar,IT Services,Roof Deck,Café,Bus Stop,Hardware Store,Lake,Women's Store,Middle Eastern Restaurant,Beer Garden,Food Court,Japanese Restaurant,Jewelry Store,Cupcake Shop,Other Nightlife,Bookstore,Art Museum,Storage Facility,Hookah Bar,Supplement Shop,Buffet,History Museum,Halal Restaurant,French Restaurant,Mediterranean Restaurant,Ethiopian Restaurant,Big Box Store,Coffee Shop,Pie Shop,Photography Studio,Video Game Store,Smoothie Shop,Tennis Court,Farmers Market,Rock Club,Hotel Bar,Indie Theater,Convenience Store,Theater,Falafel Restaurant,Road,Ramen Restaurant,Harbor / Marina,Baby Store,Historic Site,Outlet Store,American Restaurant,Tiki Bar,Supermarket,Salon / Barbershop,Beach,Cafeteria,Hot Dog Joint,Seafood Restaurant,Turkish Restaurant,Bike Trail,Animal Shelter,Hotel,Bridal Shop,Lawyer,Persian Restaurant,Dry Cleaner,Taco Place,Steakhouse,Climbing Gym,Airport Terminal,Scandinavian Restaurant,Bus Line,Music Store,Speakeasy,Southern / Soul Food Restaurant,Wine Shop,Field,New American Restaurant,Sushi Restaurant,Food Stand,Indian Restaurant,Afghan Restaurant,Snack Place,Outdoor Supply Store,African Restaurant,Motorcycle Shop,Noodle House,Gift Shop,Beer Bar,Sculpture Garden,Pet Service,Korean Restaurant,Shopping Mall,Bank,Greek Restaurant,Deli / Bodega,Multiplex,Hotpot Restaurant,Indie Movie Theater,Home Service,Costume Shop,Rental Service,Wine Bar,Filipino Restaurant,Cuban Restaurant,Metro Station,Yemeni Restaurant,Gym,Music Venue,Spa,Street Art,Indoor Play Area,Furniture / Home Store,BBQ Joint,Cocktail Bar,Empanada Restaurant,North Indian Restaurant,Comfort Food Restaurant,Bubble Tea Shop,General Entertainment,Kids Store,Performing Arts Venue,Yoga Studio,Nightclub,Playground,Burger Joint,Brazilian Restaurant,Smoke Shop,Taiwanese Restaurant,College Cafeteria,Dessert Shop,Dentist's Office,Pool,Shopping Plaza,Karaoke Bar,Massage Studio,Salad Place,Szechuan Restaurant,Israeli Restaurant,Check Cashing Service,Recreation Center,Other Repair Shop,Dive Bar,Tailor Shop,Track,Gourmet Shop,Candy Store,Gastropub,Boat or Ferry,Laundromat,Arts & 

separating x and y (the relative amount of pizzerias is the value to be predicted y)

In [16]:
chicago_Y = chicago_30['Pizza Place']
chicago_X = chicago_30
del chicago_X['Pizza Place']

In [17]:
chicago_counts=chicago_X['counts']

# Part 3: training the model on Chicago

normalization

In [18]:
X = preprocessing.StandardScaler().fit(chicago_X).transform(chicago_X.astype(float))

separating train and test data

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (X, chicago_Y, test_size = 0.3, random_state = 0)

find best K for a K nearest neighbors regressor, weighting them with the distance to the predictor

In [20]:
Ks = 10
mse = np.zeros((Ks-1))
for n in range(1,Ks):
    neigh = KNeighborsRegressor(n_neighbors = n, weights='distance').fit(x_train,y_train)
    yhat=neigh.predict(x_test)
    mse[n-1] = mean_squared_error(y_test, yhat)
print ("best K is %d with MSE %f" % (mse.argmin()+1, mse.min()))

best K is 4 with MSE 0.000807


In [21]:
meanY = y_test.mean()
print ('the mean frequency of pizza places in the test set is %f, so the MSE is %f %% of the mean frequency'
      % (meanY, 100*mse.min()/meanY))

the mean frequency of pizza places in the test set is 0.033177, so the MSE is 2.431865 % of the mean frequency


which means the model has great accuracy

In [22]:
myModel = KNeighborsRegressor(n_neighbors=5, weights='distance').fit(X,chicago_Y)

# Part 4: predictions on the new city and comparisons

prepare the dataset

In [23]:
ny_Y = ny_30['Pizza Place']
ny_X = ny_30
del ny_X['Pizza Place']

ny_counts=ny_X['counts']

X = preprocessing.StandardScaler().fit(ny_X).transform(ny_X.astype(float))

predict

In [24]:
y_hat = myModel.predict(X)

create dataset with all useful information

In [25]:
ny_Y = pd.DataFrame(ny_Y)

In [26]:
NY_final = pd.merge(ny_total, ny_Y, on='Neighborhood')

In [27]:
NY_final['predicted'] = y_hat

In [28]:
NY_final.head()

,Neighborhood,counts,Pizza Place,predicted
0,Allerton,35,0.114286,0.023586
1,Astoria,96,0.020833,0.000000
2,Bath Beach,47,0.042553,0.040523
3,Battery Park City,40,0.025000,0.016199
4,Bay Ridge,85,0.047059,0.000000


accuracy score

In [29]:
meanY = ny_Y.mean()
mse = mean_squared_error (ny_Y, y_hat)
print ('the mean frequency of pizza places in the ny set is %f, so the MSE is %f %% of the mean frequency'
      % (meanY, 100*mse.min()/meanY))

the mean frequency of pizza places in the ny set is 0.038435, so the MSE is 3.464654 % of the mean frequency


add differences between prediction and actual frequency to the dataset

In [30]:
NY_final['Frequency difference']=NY_final['predicted']-NY_final['Pizza Place']
NY_final['Absolute difference']=NY_final['Frequency difference']*NY_final['counts']

In [31]:
NY_final.head()

,Neighborhood,counts,Pizza Place,predicted,Frequency difference,Absolute difference
0,Allerton,35,0.114286,0.023586,-0.090700,-3.174503
1,Astoria,96,0.020833,0.000000,-0.020833,-2.000000
2,Bath Beach,47,0.042553,0.040523,-0.002030,-0.095420
3,Battery Park City,40,0.025000,0.016199,-0.008801,-0.352041
4,Bay Ridge,85,0.047059,0.000000,-0.047059,-4.000000


look for a bias in the predictions, which could be due to differences between the two cities

In [32]:
print ('The mean absolute difference between predicted amount of Pizza Places and actual amount is: %f' 
       %NY_final['Absolute difference'].mean())

The mean absolute difference between predicted amount of Pizza Places and actual amount is: -1.352538


This means New York tends to have a slightly higher relative amount of Pizza Places.
It's not a big problem: it means that the predictions are a bit pessimistic.

# Part 6: visualize the results on a Folium map and final results

add back the coordinates to the dataset

In [33]:
NY_final.set_index('Neighborhood', inplace=True)

In [34]:
ny_neighborhoods.set_index('Neighborhood', inplace=True)

In [35]:
NY = pd.merge(ny_neighborhoods, NY_final, on='Neighborhood', how='inner')

In [36]:
NY.head()

,Latitude,Longitude,counts,Pizza Place,predicted,Frequency difference,Absolute difference
Neighborhood,,,,,,,
Kingsbridge,40.881687,-73.902818,67,0.089552,0.034105,-0.055447,-3.714982
Fordham,40.860997,-73.896427,80,0.012500,0.010471,-0.002029,-0.162322
Westchester Square,40.840619,-73.842194,32,0.062500,0.024275,-0.038225,-1.223187
Belmont,40.857277,-73.888452,97,0.092784,0.017693,-0.075091,-7.283803
Pelham Bay,40.850641,-73.832074,36,0.000000,0.034484,0.034484,1.241410


visualize

In [37]:
NY.reset_index(inplace=True)

In [38]:
venues_map = folium.Map(location=[40.730610, -73.935242], zoom_start=10)

def getColor (difference):
    if difference < -1:
        return 'darkred'
    elif difference < 0:
        return 'red'
    elif difference < 1:
        return 'orange'
    elif difference < 2:
        return 'lightgreen'
    else:
        return 'darkgreen'

for i in range (0, len(NY)):
    folium.CircleMarker([NY.loc[i, 'Latitude'], NY.loc[i, 'Longitude']], 
                        radius=5, color=getColor(NY.loc[i, 'Absolute difference']),
                        popup='{} : there seems to be place for {} Pizza places more'.format(NY.loc[i, 'Neighborhood'], NY.loc[i, 'Absolute difference']),
                        fill = True, fill_color = getColor(NY.loc[i, 'Absolute difference']), fill_opacity = 0.6).add_to(venues_map)

In [39]:
venues_map

find best neighborhoods to start a pizza place in New York

In [40]:
NY.sort_values(by='Absolute difference', axis=0, ascending=False, inplace=True)

In [41]:
NY

,Neighborhood,Latitude,Longitude,counts,Pizza Place,predicted,Frequency difference,Absolute difference
90,Civic Center,40.715229,-74.005415,84,0.000000,0.031243,0.031243,2.624421
39,Yorkville,40.77593,-73.947118,100,0.010000,0.029789,0.019789,1.978907
4,Pelham Bay,40.850641,-73.832074,36,0.000000,0.034484,0.034484,1.241410
64,Howard Beach,40.654225,-73.838138,40,0.000000,0.028646,0.028646,1.145856
27,East Williamsburg,40.708492,-73.938858,67,0.000000,0.016990,0.016990,1.138352
19,Park Slope,40.672321,-73.97705,49,0.020408,0.042481,0.022073,1.081590
84,West Brighton,40.631879,-74.107182,37,0.027027,0.053457,0.026430,0.977920
63,Jackson Heights,40.751981,-73.882821,75,0.013333,0.025674,0.012341,0.925570
10,Prospect Heights,40.676822,-73.964859,59,0.016949,0.031277,0.014328,0.845356
23,Boerum Hill,40.685683,-73.983748,90,0.000000,0.008406,0.008406,0.756515
